In [3]:
## Load libraries
import sys
import os
import numpy as np
import pandas as pd
from copy import deepcopy

# Add src directory to Python path to import TaxSolver
src_path = os.path.join(os.getcwd(), 'src')
if src_path not in sys.path:
    sys.path.insert(0, src_path)

# Import TaxSolver modules
import TaxSolver as tx
from TaxSolver.data_wrangling.data_loader import DataLoader
from TaxSolver.constraints.budget_constraint import BudgetConstraint
from TaxSolver.constraints.income_constraint import IncomeConstraint
from TaxSolver.objective import BudgetObjective
from TaxSolver.data_wrangling.bracket_input import BracketInput

Load table with taxpayers into DataLoader

In [4]:
## Load data
file_path = os.path.join('..', 'data', 'example', 'simple_simul_1000.xlsx')
df_taxpayers = pd.read_excel(file_path)
df_taxpayers['hh_id'] = df_taxpayers.index
# df_taxpayers = df_taxpayers.loc[df_taxpayers['income_before_tax'] <= 125_000, :].copy()

dl = DataLoader(path=df_taxpayers, income_before_tax="income_before_tax", income_after_tax="outcome_1")

Directly loading from pd.DataFrame...
Setting 'weight' to 1 as default.
Mirror household were missing for 1000 households: set to own id


Setup TaxSolver model

In [5]:
# Initialize the model
tax_solver = tx.TaxSolver(dl.households)

Set parameter Username
Academic license - for non-commercial use only - expires 2026-03-13


Split income_before_tax into sections based on inflection points

In [6]:
BracketInput.add_split_variables_to_solver(
    tx=tax_solver,
    target_var="income_before_tax",
    inflection_points=[0, 25_000, 50_000, 75_000, 100_000, 125_000, 150_000],
    group_vars=["k_everybody"]      # or any other indicator columns
)

Add solver variables through taxrules

In [7]:
## OPTION 1:
# ## Add FlatTaxRule manually
# income_tax_flat_0_25 = tx.FlatTaxRule(
#     name="income_before_tax_k_everybody_0_25000",
#     var_name="income_before_tax_k_everybody_0_25000",
#     # k_group_var="k_everybody",
#     marginal_pressure=True,
#     ub=1,
#     lb=0,
# )

# ## Add FlatTaxRule manually
# income_tax_flat_25_50 = tx.FlatTaxRule(
#     name="income_before_tax_k_everybody_25000_50000",
#     var_name="income_before_tax_k_everybody_25000_50000",
#     # k_group_var="k_everybody",
#     marginal_pressure=True,
#     ub=1,
#     lb=0,
# )

## OPTION 2:
## Add BracketRule
income_tax = tx.BracketRule(
    name="income_before_tax_k_everybody",
    var_name="income_before_tax",
    k_group_var="k_everybody",
    ub=1,
    lb=0,
)

universal_benefit = tx.BenefitRule(
    name="universal_benefit",
    var_name="k_everybody",
)

# ## OPTION 1:
# tax_solver.add_rules([income_tax_flat_0_25, income_tax_flat_25_50, universal_benefit])

## OPTION 2:
tax_solver.add_rules([income_tax, universal_benefit])

Add constraints

In [8]:
income_constraint = IncomeConstraint(0.0001, dl.households.values())
budget_constraint = BudgetConstraint(
    "All_households", dl.households.values(), 0, 0
)

tax_solver.add_constraints([income_constraint, budget_constraint])

Add objective

In [10]:
tax_solver.add_objective(BudgetObjective(budget_constraint))

Solve

In [11]:
# Solve the system
tax_solver.solve()

Current tax balance All_households: -19991111
New Maximum: -19991111
New Minimum: -19991111
Going to solve!
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 24.5.0 24F74)

CPU model: Apple M4 Max
Thread count: 16 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 16004 rows, 14016 columns and 32677 nonzeros
Model fingerprint: 0xd539ba65
Model has 7 simple general constraints
  7 INDICATOR
Variable types: 14009 continuous, 7 integer (7 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+04]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+06]
  RHS range        [1e+00, 2e+07]
  GenCon coe range [1e+00, 1e+00]
Presolve removed 15004 rows and 13010 columns
Presolve time: 0.03s
Presolved: 1000 rows, 1006 columns, 6663 nonzeros
Variable types: 1006 continuous, 0 integer (0 binary)

Root relaxation: objective -1.999111e+07, 18 iterations, 0.00 seconds (0.01 work units)

    Nodes    |    Current Node    |     Obj

Inspect rates

In [12]:
tax_solver.rules_and_rates_table()

,rule_name,rule_type,var_name,rate,b,weight
0,income_before_tax_k_everybody__income_before_t...,FlatTaxRule,income_before_tax_k_everybody_0_25000,0.107114,1.0,1
1,income_before_tax_k_everybody__income_before_t...,FlatTaxRule,income_before_tax_k_everybody_25000_50000,0.200076,1.0,1
2,income_before_tax_k_everybody__income_before_t...,FlatTaxRule,income_before_tax_k_everybody_50000_75000,0.300070,1.0,1
3,income_before_tax_k_everybody__income_before_t...,FlatTaxRule,income_before_tax_k_everybody_75000_100000,0.400060,1.0,1
4,income_before_tax_k_everybody__income_before_t...,FlatTaxRule,income_before_tax_k_everybody_100000_125000,0.500050,1.0,1
5,income_before_tax_k_everybody__income_before_t...,FlatTaxRule,income_before_tax_k_everybody_125000_150000,0.500050,1.0,1
6,universal_benefit,BenefitRule,k_everybody,175.995362,1.0,1
